In [ ]:
import os, json
import yaml


# root = '/private/home/delong/workspace/subobjects-VLM/runs/pixmo_cap/Llama-3_2-1B'
# steps = 8283


# root = '/private/home/delong/workspace/subobjects-VLM/runs/clevr_caption/SmolLM2-135M-Instruct/'
root = '/private/home/delong/workspace/subobjects-VLM/runs/clevr_caption/SmolLM2-360M-Instruct/'
steps = 4080

cmaps = ['Reds', 'Greens', 'Purples', 'Blues']
families = ['patch', 'superpixel', 'directsam', 'panoptic']

all_results = []
for family in families:
    runs = os.listdir(root + family)
    results = []
    for run in runs:
        if run=='1206-1617-patch_4_per_side_random(256t-384px)-vae(384px)':
            continue
        print(run)
        trainer_states = os.path.join(root, family, run, f'runs/checkpoint-{steps}/trainer_state.json')
        if not os.path.exists(trainer_states):
            continue
        trainer_states = json.load(open(trainer_states))["log_history"]

        args = os.path.join(root, family, run, 'runs/args.yaml')
        args = yaml.load(open(args), Loader=yaml.FullLoader)
        family = args['visual_tokenizer_config'].split('/')[-2]
        results.append((family, args, trainer_states))
    print(len(results))
    all_results.append(results)

In [ ]:


def sort_key(result):
    visual_tokenizer_config = result[1]['visual_tokenizer_config']
    if '/patch' in visual_tokenizer_config:
        return f"{int(visual_tokenizer_config.split('patch_')[-1].split('_')[0]):2}"
    else:
        return visual_tokenizer_config

# Sort the results based on the visual_tokenizer_config
for results in all_results:
    results = sorted(results, key=sort_key)

    for result in results:
        family, args, trainer_states = result
        print(family, args['visual_tokenizer_config'])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

color_offset = 10
plt.figure(figsize=(20, 15))
for j, family in enumerate(families):
    
    family_results = all_results[j]

    colors = plt.cm.get_cmap(cmaps[j], len(family_results)+color_offset)

    for i, (family, args, trainer_states) in enumerate(family_results):
        # print(args.keys())

        steps = [x['step'] for x in trainer_states]
        loss = [x['loss'] for x in trainer_states]
        loss_smoothed = np.convolve(loss, np.ones(32)/32, mode='valid')
        pad_size = (len(loss) - len(loss_smoothed)) // 2

        family, tokenizer = args['visual_tokenizer_config'].split('/')[-2:]
        tokenizer = tokenizer.replace('.json', '')

        max_visual_tokens = args['max_visual_tokens']
        tokenizer = f"{tokenizer} ({max_visual_tokens})"

        # plt.plot(steps, loss, color=colors(i+5), alpha=0.5, linewidth=1)
        plt.plot(
            steps[pad_size:-pad_size-1], loss_smoothed, 
            color=colors(len(family_results) + color_offset // 2), 
            # color=colors(i+color_offset), 
            alpha=0.5, linewidth=2, label=tokenizer
            )

    plt.legend()
plt.ylim(0,0.8) # CLEVR 135M
plt.xlim(0, len(steps))
plt.show()